Quick notebook to prepae examples (not the tfrecord kind) from Project Gutenborgs Bible text

In [1]:
import re

In [2]:
txt = open('./pg10.txt').read()

In [3]:
split_reg = re.compile('\n{4}')

# split the raw text of the bible into books

In [4]:
books = split_reg.split(txt)

In [11]:
books = books[4:] # The first 4 are preamble from project gutenberg

Split each book into the name of the book and its text

In [34]:
book_verse_spliter = re.compile('\n{3}')
book,verses = book_verse_spliter.split(books[1])

In [35]:
book

'\nThe Second Book of Moses:  Called Exodus'

In [19]:
verses[:100]

'1:1 In the beginning God created the heavens and the earth.\n\n1:2 And the earth was without form, and'

Split all the text in one book into verses. Make it a dict of chapter, verse and text

In [24]:
verses_splitter = re.compile('(?P<chapter>\d+)\:(?P<verse>\d+)(?P<text>.+)',flags=re.MULTILINE)

In [25]:
gen = verses_splitter.finditer(verses)

In [29]:

next(gen).groupdict()

{'chapter': '1',
 'text': ' And God saw the light, that it was good: and God divided the light',
 'verse': '4'}

Lets run all of that on the entire bible

In [62]:
examples= []

for num,book in enumerate(books):
    try:
        book_name, book_text = book_verse_spliter.split(book)
        book_name = book_name.strip()
        for verse_regex_match in verses_splitter.finditer(book_text):
            example = verse_regex_match.groupdict()
            example.update({"book":book_name,"book_id":num,"text":example["text"].strip()})
            examples.append(example)
    except:
        #Hack, the above only gets us to psalms
        pass

In [68]:
examples[:3]

[{'book': 'The First Book of Moses:  Called Genesis',
  'book_id': 0,
  'chapter': '1',
  'text': 'In the beginning God created the heavens and the earth.',
  'verse': '1'},
 {'book': 'The First Book of Moses:  Called Genesis',
  'book_id': 0,
  'chapter': '1',
  'text': 'And the earth was without form, and void; and darkness was upon',
  'verse': '2'},
 {'book': 'The First Book of Moses:  Called Genesis',
  'book_id': 0,
  'chapter': '1',
  'text': 'And God said, Let there be light: and there was light.',
  'verse': '3'}]

Lets save it

In [81]:
import pandas as pd
D = pd.DataFrame(examples)

In [83]:
D.book_id.max()

214

In [67]:
pickle.dump(examples,open('./bible_data.pkl','wb'))

# Now we make them into TF records

In [70]:
import tensorflow as tf
from preppy import Preppy
class BibPreppy(Preppy):
    '''
    We'll slightly extend to way we right tfrecords to store the id of the book it came from
    '''
    def sequence_to_tf_example(self,sequence,book_id):
        id_list = self.sentance_to_id_list(sequence)
        ex = tf.train.SequenceExample()
        # A non-sequential feature of our example
        sequence_length = len(sequence)
        ex.context.feature["length"].int64_list.value.append(sequence_length)
        ex.context.feature["book_id"].int64_list.value.append(book_id)
        # Feature lists for the two sequential features of our example
        fl_tokens = ex.feature_lists.feature_list["tokens"]

        for token in id_list:
            fl_tokens.feature.add().int64_list.value.append(token)

        return ex
    def parse(ex):
        '''
        Explain to TF how to go froma  serialized example back to tensors
        :param ex:
        :return:
        '''
        context_features = {
            "length": tf.FixedLenFeature([], dtype=tf.int64),
            "book_id": tf.FixedLenFeature([], dtype=tf.int64)
        }
        sequence_features = {
            "tokens": tf.FixedLenSequenceFeature([], dtype=tf.int64),
        }

        # Parse the example (returns a dictionary of tensors)
        context_parsed, sequence_parsed = tf.parse_single_sequence_example(
            serialized=ex,
            context_features=context_features,
            sequence_features=sequence_features
        )
        return {"seq":sequence_parsed["tokens"], "length": context_parsed["length"], 
                "book_id": context_parsed["book_id"]}



In [72]:
import random
random.shuffle(examples)
val,train = examples[:3000], examples[3000:]

In [75]:
BP =BibPreppy(tokenizer_fn=list) #Charecter level tokenization
for (data,path) in [(val,'./val.tfrecord'),(train,'./train.tfrecord')]:
    with open(path,'w') as f:
        writer = tf.python_io.TFRecordWriter(f.name)
    for example in data:
        record = BP.sequence_to_tf_example(sequence=example["text"],book_id=example["book_id"])
        writer.write(record.SerializeToString())


In [76]:
BP.update_reverse_vocab()

In [77]:
pickle.dump(BP,open('./preppy.pkl','wb'))

AttributeError: Can't pickle local object 'Preppy.__init__.<locals>.<lambda>'

In [78]:
len(BP.vocab)

74

In [ ]:
boo